# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use the `bedrock-agent` and the `bedrock-agent-runtime` boto3 clients to:
- create an agent
- create and action group
- associate the agent with the action group and prepare the agent
- create an agent alias
- invoke the agent

We will use Bedrock's Claude v2.1 using the Boto3 API. 

**Note:** *This notebook can be run within or outside of AWS environment.*

#### Pre-requisites
This notebook requires permissions to: 
- create and delete Amazon IAM roles
- create, update and invoke AWS Lambda functions 
- create, update and delete Amazon S3 buckets 
- access Amazon Bedrock 

If you are running this notebook without an Admin role, make sure that your role include the following managed policies:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess


#### Context
We will demonstrate how to create and invoke an agent for Bedrock using the Boto3 SDK

#### Use case
For this notebook, we will be using an agent to create sql queries to answer questions the user is asking in natural language. The agent will create the sql query then get the response from the database to accurately respond to the users question. The following  diagram depicts a high-level architecture of this solution.

![sequence-flow-agent](images/text-to-sql-architecture-Athena.png)


The Agent created can handle the follow tasks:
- Get Database Schema
- Run SQL Query 

In [5]:
upgrade_output = !pip install --upgrade pip
install_boto3_output = !pip install boto3

In [7]:
from dependencies.config import *


In [8]:
# Set AWS_PROFILE environment variable
os.environ['AWS_PROFILE'] = '456667773660-virginia'


In [44]:

!python ./dependencies/build_infrastructure.py

[2024-02-19 14:37:35,278] p79338 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
s3://text-2-sql-agent-us-east-1-456667773660/db/
TheHistoryOfBaseball
AccountID:  456667773660
arn:aws:iam::456667773660:user/jpedram
AWSGlueServiceRole
s3://text-2-sql-agent-us-east-1-456667773660/data/TheHistoryofBaseball/
unzip_data()... finished
upload_data() ... finished
aws s3 sync ./data/extracted/ s3://text-2-sql-agent-us-east-1-456667773660/data
upload: data/extracted/TheHistoryofBaseball/hall_of_fame/hall_of_fame.csv to s3://text-2-sql-agent-us-east-1-456667773660/data/TheHistoryofBaseball/hall_of_fame/hall_of_fame.csv
upload: data/extracted/TheHistoryofBaseball/player_award_vote/player_award_vote.csv to s3://text-2-sql-agent-us-east-1-456667773660/data/TheHistoryofBaseball/player_award_vote/player_award_vote.csv
upload: data/extracted/TheHistoryofBaseball/player_award/player_award.csv to s3://text-2-sql-agent-us-east-1-456667773660/data/TheHistoryofB

In [12]:
from dependencies.config import *

list_agent=bedrock_agent_client.list_agents()['agentSummaries']
#print(list_agent)
#print(agent_name)
# Search f
agent_id = next((agent['agentId'] for agent in list_agent if agent['agentName'] == agent_name), None)

print(agent_id)

response = bedrock_agent_client.list_agent_aliases(
    agentId=agent_id,
)
response['agentAliasSummaries']
agentAliasId=next((agent['agentAliasId'] for agent in response['agentAliasSummaries'] if agent['agentAliasName'] == agent_alias_name), None)
agent_alias_id=agentAliasId
print(agent_alias_id)

None


ParamValidationError: Parameter validation failed:
Invalid type for parameter agentId, value: None, type: <class 'NoneType'>, valid types: <class 'str'>

In [37]:
# ### Invoke Agent
# Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.
query_to_agent = """What was Babe Ruth's salary in 1930?"""

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False


# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=query_to_agent,
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)
logger.info(pprint.pprint(agentResponse))

[2024-02-19 14:35:20,378] p71419 {1176191262.py:20} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 19 Feb 2024 19:35:20 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '04647810-cf5e-11ee-a8ef-acde48001122',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '755146d8-97c9-41b6-a48f-306902a36c6f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '755146d8-97c9-41b6-a48f-306902a36c6f',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f93a259df40>,
 'contentType': 'application/json',
 'sessionId': '04647810-cf5e-11ee-a8ef-acde48001122'}


In [38]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-02-19 14:35:22,692] p71419 {<timed exec>:11} INFO - {
  "agentId": "3QFLAQUPYF",
  "agentAliasId": "G6PNXG8B3Y",
  "sessionId": "04647810-cf5e-11ee-a8ef-acde48001122",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "traceId": "755146d8-97c9-41b6-a48f-306902a36c6f-0",
        "text": "You are an expert database querying assistant that can create simple and complex SQL queries to get \nthe answers to questions about baseball players that you are asked. You first need to get the schemas for the table in the database to then query the \ndatabase tables using a sql statement then respond to the user with the answer to their question and\nthe sql statement used to answer the question. Use the getschema tool first to understand the schema\nof the table then create a sql query to answer the users question.\nHere is an example to query the table <example>SELECT * FROM thehistoryofbaseball.players LIMIT 10;</example> Do not use \nquotes for the table name.

CPU times: user 19.6 ms, sys: 6.4 ms, total: 26 ms
Wall time: 21 s


In [39]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Babe Ruth's salary in 1930 was $80,000. To find this, I queried the salary table with the SQL statement: 
SELECT player_id, year, salary FROM salary WHERE player_id = 'ruthba01' AND year = 1930


In [40]:
#Create function to invoke agent
def invoke_agent(query):
    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = False
    end_session:bool = False
 

    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    event_stream = agentResponse['completion']
    print("Fetching answer...")
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



In [11]:
invoke_agent("How many MVP awards did Mickey Mantle win?")

NameError: name 'invoke_agent' is not defined

In [13]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")


Fetching answer...


[2024-02-19 13:32:11,104] p71419 {1394403490.py:24} INFO - Final answer ->
Nolan Ryan was inducted into the Baseball Hall of Fame in 1999. To find this, I queried the hall_of_fame table to get the yearid for Nolan Ryan where he was inducted as a Player. The SQL query was:

SELECT yearid FROM thehistoryofbaseball.hall_of_fame WHERE player_id = 'ryanno01' AND category = 'Player'


In [42]:
invoke_agent("In what year did Hank Aaron hit the most home runs?")


Fetching answer...


[2024-02-19 14:36:51,168] p71419 {1394403490.py:24} INFO - Final answer ->
The SQL query to find the year when Hank Aaron hit the most home runs is:

SELECT p.name_first, p.name_last, s.year, MAX(s.salary) AS max_salary
FROM player AS p
INNER JOIN salary AS s ON s.player_id = p.player_id  
WHERE p.name_first = 'Hank' AND p.name_last = 'Aaron'
GROUP BY p.name_first, p.name_last, s.year
ORDER BY max_salary DESC
LIMIT 1;

The query results show that Hank Aaron hit the most home runs in 1973.


In [120]:
#This query requires a join of two tables to be able to answer the question
invoke_agent("What player has received the most All-Star Game selections?")

Fetching answer...


[2024-02-19 16:13:56,326] p1383 {1394403490.py:24} INFO - Final answer ->
The player who has received the most All-Star Game selections is Babe Ruth with 32 selections. The SQL query used was:

SELECT p.name_first, p.name_last, COUNT(*) AS selections
FROM player_award pa
INNER JOIN player p ON pa.player_id = p.player_id  
WHERE pa.award_id LIKE '%All-Star%'
GROUP BY p.name_first, p.name_last
ORDER BY selections DESC
LIMIT 1;


In [ ]:
# This will stop the notebook's execution and wait for the user to press Enter
input("Press Enter to continue...")


## Clean up (optional)

The next steps are optional and delete the infrustcture we built. 



In [45]:
!python ./dependencies/clean.py

[2024-02-19 16:59:08,541] p94161 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
s3://text-2-sql-agent-us-east-1-456667773660/db/
TheHistoryOfBaseball
Crawler 'TheHistoryOfBaseball' deleted successfully.
Table 'hall_of_fame' deleted successfully.
Table 'player' deleted successfully.
Table 'player_award' deleted successfully.
Table 'player_award_vote' deleted successfully.
Table 'salary' deleted successfully.
Database 'thehistoryofbaseball' deleted successfully.
OMU4YF4ZA7
[{'actionGroupId': 'CWAN6CRWYA', 'actionGroupName': 'QueryAthenaActionGroup', 'actionGroupState': 'ENABLED', 'description': 'Actions for getting the database schema and querying the Athena database', 'updatedAt': datetime.datetime(2024, 2, 19, 19, 41, 20, 466757, tzinfo=tzutc())}]
CWAN6CRWYA
<class 'list'>
text-2-sql-agent-us-east-1-456667773660
arn:aws:lambda:us-east-1:456667773660:function:text-2-sql-agent-us-east-1-456667773660
Policy 'text-2-sql-agent-allow-us-east-1-4


## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You